In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
df = pd.read_csv(path, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [31]:
X = df.iloc[:,2:].values

In [32]:
y=df.iloc[:,1]

In [33]:
y = np.where(y=='M', 1, 0)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=1)

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [37]:
from sklearn.pipeline import make_pipeline

In [38]:
pipe_line = make_pipeline(StandardScaler(), PCA(n_components=2), LogisticRegression(random_state=1))

In [39]:
type(pipe_line)

sklearn.pipeline.Pipeline

In [40]:
pipe_line.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])

In [46]:
p_pred = pipe_line.predict(X_test)

In [47]:
pipe_line.score(X_test,y_test)

0.956140350877193

In [48]:
from sklearn.metrics import confusion_matrix

In [49]:
confusion_matrix(y_test,p_pred)

array([[71,  1],
       [ 4, 38]], dtype=int64)

In [50]:
from sklearn.model_selection import cross_val_score

In [58]:
scores = cross_val_score(estimator=pipe_line,X=X_train, y=y_train, cv=10, n_jobs=-1)

In [59]:
scores

array([0.93478261, 0.93478261, 0.95652174, 0.95652174, 0.93478261,
       0.95555556, 0.97777778, 0.93333333, 0.95555556, 0.95555556])

In [60]:
np.mean(scores)

0.9495169082125603

In [54]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
best_score = 0
for k in range(1,10): 
    scores_tree = cross_val_score( estimator=DecisionTreeClassifier(max_depth=k, random_state=1), X=X_train, y=y_train, cv=5, n_jobs=-1) 
    score = np.mean(scores_tree) 
    print('depth =', k, 'accuracy =', score) 
    if score > best_score: 
        best_score = score
        best_parameters = {'max_depth':k}

print(best_score) #0.9407
print()
print(best_parameters) #{'max_depth': 4}

depth = 1 accuracy = 0.8813186813186814
depth = 2 accuracy = 0.9318681318681319
depth = 3 accuracy = 0.9296703296703297
depth = 4 accuracy = 0.9406593406593406
depth = 5 accuracy = 0.9384615384615385
depth = 6 accuracy = 0.9362637362637363
depth = 7 accuracy = 0.9406593406593406
depth = 8 accuracy = 0.9406593406593406
depth = 9 accuracy = 0.9406593406593406
0.9406593406593406

{'max_depth': 4}


In [56]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': range(1, 10)}
gscv = GridSearchCV( estimator=DecisionTreeClassifier(random_state=1), param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gscv.fit(X_train, y_train)

print(gscv.best_score_) #0.9407
print(gscv.best_params_) #{'max_depth': 4}

0.9406593406593406
{'max_depth': 4}


In [57]:
from sklearn.svm import SVC

pipe_svc = make_pipeline(StandardScaler(), SVC(kernel='rbf', random_state=1))
p_list = [0.001,0.01,0.1,1.0,10.0,100.0]
param_grid = {'svc__C':p_list, 'svc__gamma':p_list}
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_params_) #{'svc__C': 10.0, 'svc__gamma': 0.01}
print(gs.score(X_test, y_test)) #0.9739

{'svc__C': 10.0, 'svc__gamma': 0.01}
0.9736842105263158
